In [1]:
from aequilibrae import Project
import pandas as pd
import rasterio

In [2]:
from functions.raster_to_dataframe import raster_to_df
from functions.country_main_area import get_main_area
from functions.check_population import check_pop
from functions.preview_raster import preview_raster

## Model place (country) and population raster definition
(the raster must be in the same folder as this notebook)

In [3]:
model_place = 'Andorra'
pop_file = 'andorra_population.tif'

## Loading project and getting country main area

In [4]:
model_fldr = f'../{model_place}'

In [5]:
project = Project()
project.open(model_fldr)

In [6]:
main_area = get_main_area(project)

## Processing the raster population information and transforming it into a dataframe
(slow for large datasets)

In [ ]:
df = raster_to_df(rasterio.open(pop_file), main_area)

## Comparing the total vectorized population to a World Bank source

In [ ]:
check_pop(model_place, df)

## Creating a new table inside the model for the vectorized population

In [ ]:
%%time
# Creating a new table inside the model and inserting the vectorized population there
conn = project.conn
df.to_sql('raw_population', conn, if_exists='replace', index=False)
conn.execute("select AddGeometryColumn( 'raw_population', 'geometry', 4326, 'POINT', 'XY', 1);")

## Creating point geometries from the population latitude and longitude coordinates

In [ ]:
%%time
conn.execute("UPDATE raw_population SET Geometry=MakePoint(longitude, latitude, 4326)")
conn.commit()

In [ ]:
project.close()